<a href="https://colab.research.google.com/github/vivekkhare22/VK_AI_Repository/blob/main/I_FS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================================
# 📘 IFS AI Functional Spec & Test Case Generator
# Tools: Google Colab + Google Sheets + OpenAI GPT
# ======================================================

# Step 1️⃣: Connect Google Colab with Google Account
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

print("✅ Connected to Google Account successfully!")


In [ ]:
# ------------------------------------------------------
# Step 2️⃣: Open your Google Sheet
SHEET_NAME = "IFS_AI_Spec_Generator"   # <-- replace with your actual sheet name if different
req_ws = gc.open(SHEET_NAME).worksheet("Requirements")
out_ws = gc.open(SHEET_NAME).worksheet("AI_Output")

print("✅ Connected to Google Sheet:", SHEET_NAME)

In [ ]:
# ======================================================
# 📘 IFS AI Functional Spec & Test Case Generator
# Tools: Google Colab + Google Sheets + OpenAI GPT
# ======================================================

!pip install openai --quiet

import datetime
import time
from openai import OpenAI
from google.colab import userdata

# ✅ Get OpenAI API key from userdata (set via Colab > ⚙️ > User Secrets)
openai_api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

print("✅ OpenAI Client Connected")

# ------------------------------------------------------
# Step 4️⃣: Read all requirements
data = req_ws.get_all_records()

def generate_spec_and_testcases(module, summary, expected):
    prompt = f"""
    You are an expert IFS ERP Functional Consultant.
    Based on the following requirement, generate a detailed Functional Specification
    and 5 Test Cases in professional ERP format.

    Module: {module}
    Requirement Summary: {summary}
    Expected Outcome: {expected}

    Output Format:
    1. Functional Overview
    2. Business Logic / Validation Rules
    3. Configuration or Customization Details
    4. Test Scenarios (in table format: Test Case ID, Steps, Expected Result)
    """

    response = client.responses.create(
        model="gpt-4.1-mini",
        input=prompt,
    )

    return response.output_text

# ------------------------------------------------------
# Step 5️⃣: Process Pending Requirements with On-Screen Preview
for i, row in enumerate(data, start=2):
    try:
        status = row.get("Status", "").strip().lower()
        if status == "sent to ai":
            req_id = row.get("Requirement ID", "")
            module = row.get("Module", "")
            summary = row.get("Requirement Summary", "")
            expected = row.get("Expected Outcome", "")

            print("\n" + "="*90)
            print(f"🚀 Processing Requirement ID: {req_id or f'Row {i}'}")
            print(f"📘 Module: {module}")
            print(f"🧾 Summary: {summary}")
            print("-"*90)

            result = generate_spec_and_testcases(module, summary, expected)

            # 🔍 Preview output in Colab
            print("🧠 AI Generated Output:")
            print(result)
            print("="*90)

            # Pause briefly to respect API rate limits
            time.sleep(1)

            # Append output to "AI_Output"
            out_ws.append_row([
                req_id,
                result,
                "Auto-generated Test Cases included above",
                datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "AI"
            ])

            # ✅ Update status in Requirements sheet
            req_ws.update_acell(f"H{i}", "Completed")
            print(f"✅ Status updated to 'Completed' for {req_id or f'Row {i}'}")

    except Exception as e:
        print(f"❌ Error processing row {i}: {e}")

print("\n🎉 All 'Sent to AI' requirements processed successfully!")
